<a href="https://colab.research.google.com/github/aditya0811/ML_Projects/blob/master/Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a follow up of the dataset I prepared after using New York Taxi dataset from kaggle which is in the notebook.
https://github.com/aditya0811/ML_Projects/blob/master/New_York_Taxi_fare_Analysis/NYC_Taxi_Fare_Analysis.ipynb

# RESULTS


*   Using Random Forest regressor with default parameters I Mean Squared error was 173
*   Using Random Forest regressor cross validation accuracy was 79.48
* Using Grid Search (for finding best parameters) I got MSE around 14
* Use simple stacked models to predict fare price.I used here averaged models(XGBoost,ENet,GradientBoosting) which individually gave RMSE value as (4.4437,9.6255,4.29) and after averaging RMSE 5.1439


#Data preparation

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [0]:
df_train =  pd.read_csv('drive/My Drive/training.csv', parse_dates=["pickup_datetime"],nrows=50000)

In [0]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 15 columns):
Unnamed: 0           50000 non-null int64
key                  50000 non-null object
fare_amount          50000 non-null float64
pickup_datetime      50000 non-null datetime64[ns]
pickup_longitude     50000 non-null float64
pickup_latitude      50000 non-null float64
dropoff_longitude    50000 non-null float64
dropoff_latitude     50000 non-null float64
passenger_count      50000 non-null float64
H_Distance           50000 non-null float64
Year                 50000 non-null float64
Month                50000 non-null float64
Date                 50000 non-null float64
Day of Week          50000 non-null float64
Hour                 50000 non-null float64
dtypes: datetime64[ns](1), float64(12), int64(1), object(1)
memory usage: 5.7+ MB


In [0]:
df_train.head(5)

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,H_Distance,Year,Month,Date,Day of Week,Hour
0,0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1.0,1.030764,2009.0,6.0,15.0,0.0,17.0
1,1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1.0,8.450134,2010.0,1.0,5.0,1.0,16.0
2,2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2.0,1.389525,2011.0,8.0,18.0,3.0,0.0
3,3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1.0,2.799270,2012.0,4.0,21.0,5.0,4.0
4,4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1.0,1.999157,2010.0,3.0,9.0,1.0,7.0


In [0]:
df_train.drop(['Unnamed: 0','pickup_datetime','key'], axis=1, inplace=True)

In [0]:
df_train.columns

Index(['fare_amount', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
       'H_Distance', 'Year', 'Month', 'Date', 'Day of Week', 'Hour'],
      dtype='object')

In [0]:
month_name = {1: 'Jan',2: 'Feb',3: 'Mar',4: 'April',5: 'May',6: 'Jun',7: 'July',8: 'Aug',9: 'Sept',10: 'Oct',11: 'Nov',12: 'Dec'}
day_week={0: 'Mon',1: 'Tues',2: 'Wed',3: 'Thurs',4: 'Fri',5: 'Sat',6: 'Sun'}

In [0]:
df_train['Month'].replace(month_name , inplace =True)
df_train['Day of Week'].replace(day_week,inplace=True)
df_train.head(5)

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,H_Distance,Year,Month,Date,Day of Week,Hour
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1.0,1.030764,2009.0,Jun,15.0,Mon,17.0
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1.0,8.450134,2010.0,Jan,5.0,Tues,16.0
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2.0,1.389525,2011.0,Aug,18.0,Thurs,0.0
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1.0,2.799270,2012.0,April,21.0,Sat,4.0
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1.0,1.999157,2010.0,Mar,9.0,Tues,7.0


**Renaming columns for Hours ,so that we can make difference between Hour and time columns to use One hot encoding**

In [0]:
df_train.head(5)

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,H_Distance,Year,Month,Date,Day of Week,Hour
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1.0,1.030764,2009.0,Jun,15.0,Mon,H17.0
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1.0,8.450134,2010.0,Jan,5.0,Tues,H16.0
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2.0,1.389525,2011.0,Aug,18.0,Thurs,H0.0
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1.0,2.799270,2012.0,April,21.0,Sat,H4.0
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1.0,1.999157,2010.0,Mar,9.0,Tues,H7.0


In [0]:
cols = ('Month', 'Day of Week', 
        'Hour', 'Date','Year')
for col in cols:
  df_train = pd.concat([df_train,pd.get_dummies(df_train[col], prefix=col)],axis=1)
  df_train.drop([col],axis=1, inplace=True)
  

In [0]:
df_train.columns

Index(['fare_amount', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
       'H_Distance', 'Month_April', 'Month_Aug', 'Month_Dec', 'Month_Feb',
       'Month_Jan', 'Month_July', 'Month_Jun', 'Month_Mar', 'Month_May',
       'Month_Nov', 'Month_Oct', 'Month_Sept', 'Day of Week_Fri',
       'Day of Week_Mon', 'Day of Week_Sat', 'Day of Week_Sun',
       'Day of Week_Thurs', 'Day of Week_Tues', 'Day of Week_Wed', 'Hour_H0.0',
       'Hour_H1.0', 'Hour_H10.0', 'Hour_H11.0', 'Hour_H12.0', 'Hour_H13.0',
       'Hour_H14.0', 'Hour_H15.0', 'Hour_H16.0', 'Hour_H17.0', 'Hour_H18.0',
       'Hour_H19.0', 'Hour_H2.0', 'Hour_H20.0', 'Hour_H21.0', 'Hour_H22.0',
       'Hour_H23.0', 'Hour_H3.0', 'Hour_H4.0', 'Hour_H5.0', 'Hour_H6.0',
       'Hour_H7.0', 'Hour_H8.0', 'Hour_H9.0', 'Date_1.0', 'Date_2.0',
       'Date_3.0', 'Date_4.0', 'Date_5.0', 'Date_6.0', 'Date_7.0', 'Date_8.0',
       'Date_9.0', 'Date_10.0', 'Date_11.0', 'Date_12.0', 'Date

In [0]:
print('Shape df_train: {}'.format(df_train.shape))

Shape df_train: (50000, 88)


In [0]:
df_train.head(5)

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,H_Distance,Month_April,Month_Aug,Month_Dec,...,Date_29.0,Date_30.0,Date_31.0,Year_2009.0,Year_2010.0,Year_2011.0,Year_2012.0,Year_2013.0,Year_2014.0,Year_2015.0
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1.0,1.030764,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1.0,8.450134,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2.0,1.389525,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1.0,2.799270,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1.0,1.999157,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [0]:
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train, df_train.fare_amount, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(40000, 88) (40000,)
(10000, 88) (10000,)


In [0]:
X_train.columns

Index(['fare_amount', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
       'H_Distance', 'Month_April', 'Month_Aug', 'Month_Dec', 'Month_Feb',
       'Month_Jan', 'Month_July', 'Month_Jun', 'Month_Mar', 'Month_May',
       'Month_Nov', 'Month_Oct', 'Month_Sept', 'Day of Week_Fri',
       'Day of Week_Mon', 'Day of Week_Sat', 'Day of Week_Sun',
       'Day of Week_Thurs', 'Day of Week_Tues', 'Day of Week_Wed', 'Hour_H0.0',
       'Hour_H1.0', 'Hour_H10.0', 'Hour_H11.0', 'Hour_H12.0', 'Hour_H13.0',
       'Hour_H14.0', 'Hour_H15.0', 'Hour_H16.0', 'Hour_H17.0', 'Hour_H18.0',
       'Hour_H19.0', 'Hour_H2.0', 'Hour_H20.0', 'Hour_H21.0', 'Hour_H22.0',
       'Hour_H23.0', 'Hour_H3.0', 'Hour_H4.0', 'Hour_H5.0', 'Hour_H6.0',
       'Hour_H7.0', 'Hour_H8.0', 'Hour_H9.0', 'Date_1.0', 'Date_2.0',
       'Date_3.0', 'Date_4.0', 'Date_5.0', 'Date_6.0', 'Date_7.0', 'Date_8.0',
       'Date_9.0', 'Date_10.0', 'Date_11.0', 'Date_12.0', 'Date

In [0]:
X_test.columns

Index(['fare_amount', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
       'H_Distance', 'Month_1.0', 'Month_2.0', 'Month_3.0', 'Month_4.0',
       'Month_5.0', 'Month_6.0', 'Month_7.0', 'Month_8.0', 'Month_9.0',
       'Month_10.0', 'Month_11.0', 'Month_12.0', 'Day of Week_0.0',
       'Day of Week_1.0', 'Day of Week_2.0', 'Day of Week_3.0',
       'Day of Week_4.0', 'Day of Week_5.0', 'Day of Week_6.0', 'Hour_H0.0',
       'Hour_H1.0', 'Hour_H10.0', 'Hour_H11.0', 'Hour_H12.0', 'Hour_H13.0',
       'Hour_H14.0', 'Hour_H15.0', 'Hour_H16.0', 'Hour_H17.0', 'Hour_H18.0',
       'Hour_H19.0', 'Hour_H2.0', 'Hour_H20.0', 'Hour_H21.0', 'Hour_H22.0',
       'Hour_H23.0', 'Hour_H3.0', 'Hour_H4.0', 'Hour_H5.0', 'Hour_H6.0',
       'Hour_H7.0', 'Hour_H8.0', 'Hour_H9.0', 'Date_1.0', 'Date_2.0',
       'Date_3.0', 'Date_4.0', 'Date_5.0', 'Date_6.0', 'Date_7.0', 'Date_8.0',
       'Date_9.0', 'Date_10.0', 'Date_11.0', 'Date_12.0', 'Date_13.

In [0]:
X_train = X_train.iloc[:,X_train.columns!='fare_amount']
X_test = X_test.iloc[:,X_test.columns!='fare_amount']

#Modelling

In [0]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

#### Using  random forest

In [0]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [0]:
rf.fit(X_train, y_train)
rf_predict = rf.predict(X_test)
#print(rf_predict)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
print("Mean squared error: %.2f" % np.mean((rf_predict - y_test) ** 2))
from sklearn.metrics import explained_variance_score
explained_variance_score(y_test, rf_predict, multioutput='uniform_average')

Mean squared error: 173.98


-0.8175064303587467

#### Random forest with crossvalidation (only on training dataset)

In [0]:
#Using cross validation
from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator= rf,X=X_train,y=y_train,cv=10)

In [0]:
accuracies.mean()

0.7948409503540399

In [0]:
accuracies.std()

0.04610456256713789

**Using Grid Search to find best parameters**

**Methodologies used for modelling**


*   BOOTSTRAP-The bootstrap method is a statistical technique for estimating quantities about a population by averaging estimates from multiple small data samples.
    

    *   Here the estimation can be predicted with confidence interval which is not available in cross validation
    *   We must choose size of sample and number of repeats.
* know about parameters


In [0]:
#Using gridsearch
from sklearn.model_selection import GridSearchCV
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4],
    'min_samples_split': [8, 10],
    'n_estimators': [100, 200]
}

In [0]:
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [0]:
grid_search=grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:  5.5min finished


In [0]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 200}

In [0]:
rf_bestparam=RandomForestRegressor(n_estimators=200, max_depth=80, min_samples_split=8, min_samples_leaf=3, max_features=3, bootstrap=True,random_state=42)

In [0]:
rf_bestparam.fit(X_train, y_train)
rf_bestparam_predict = rf.predict(X_test)
print("Mean squared error: %.2f" % np.mean((rf_bestparam_predict - y_test) ** 2))
from sklearn.metrics import explained_variance_score
explained_variance_score(y_test,rf_bestparam_predict, multioutput='uniform_average')

Mean squared error: 16.28


0.824879817019208

### Averaging models

In [0]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [0]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [0]:
y = averaged_models.predict(X_test)
print("Mean squared error: %.2f" % np.mean((y - y_test) ** 2))
explained_variance_score(y_test, y, multioutput='uniform_average')

**[Elastic Net -Uses L1 and L2 correction terms](https://medium.com/@jayeshbahire/lasso-ridge-and-elastic-net-regularization-4807897cb722)**
Use of L1 penalty function has several limitations.[1] For example, in the "large p, small n" case (high-dimensional data with few examples), the LASSO selects at most n variables before it saturates. Also if there is a group of highly correlated variables, then the LASSO tends to select one variable from a group and ignore the others. To overcome these limitations, the elastic net adds a quadratic part to the penalty, which when used alone is ridge regression (known also as Tikhonov regularization). 

In [0]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [0]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


ElasticNet score: 9.6255 (0.2400)



In [0]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [0]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Gradient Boosting score: 4.2946 (0.3567)



In [0]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [0]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Xgboost score: 4.4437 (0.1767)



In [0]:
averaged_models = AveragingModels(models = (ENet, GBoost, model_xgb))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


 Averaged base models score: 5.1439 (0.2506)

